In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json


In [28]:
def fetch_job_description(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to fetch URL: {url}")
        return ""
    soup = BeautifulSoup(response.content, "html.parser")
    return soup.get_text(separator=" ", strip=True)


In [29]:
fetch_job_description("https://www.linkedin.com/jobs/view/junior-data-scientist-at-louis-vuitton-4184288621/?position=1&pageNum=0&refId=HgU79nN86L5X1M%2BurGd7EQ%3D%3D&trackingId=mw0LHARTIwNRAz%2F3Klz8VQ%3D%3D")

'Louis Vuitton hiring Junior Data Scientist in New York, NY | LinkedIn LinkedIn and 3rd parties use essential and non-essential cookies to provide, secure, analyze and improve our Services, and to show you relevant ads (including professional and job ads ) on and off LinkedIn. Learn more in our Cookie Policy . Select Accept to consent or Reject to decline non-essential cookies for this use. You can update your choices at any time in your settings . Accept Reject Skip to main content LinkedIn Junior Data Scientist in Milan Expand search This button displays the currently selected search type. When expanded it provides a list of search options that will switch the search inputs to match the current selection. Jobs People Learning Clear text Clear text Clear text Clear text Clear text Join now Sign in Junior Data Scientist Louis Vuitton New York, NY Apply Join or sign in to find your next job Join to apply for the Junior Data Scientist role at Louis Vuitton Not you? Remove photo First nam

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json

# Function to call Ollama API and extract job details
def extract_details_with_ollama(job_text):
    url = "http://localhost:11434/api/generate"
    prompt = f"""
Extract the following details from the job description:
- Salary
- Years of experience required
- Technical requirements
- Education level requirements

Job description:
{job_text}

Output the result as JSON with keys: "salary", "experience", "technical_requirements", "education".
"""
    payload = {"model": "llama3.2:1b", "prompt": prompt}

    response = requests.post(url, json=payload, stream=True)

    tokens = []
    for line in response.iter_lines():
        if line:
            try:
                token_data = json.loads(line.decode("utf-8"))
                token = token_data.get("response", "")
                tokens.append(token)
            except json.JSONDecodeError:
                continue

    final_answer = "".join(tokens)
    print(final_answer)
    try:
        return json.loads(final_answer)
    except json.JSONDecodeError:
        print("Error: LLM output is not valid JSON.")
        return {}

# Function to extract raw text from the job detail page
def fetch_job_description(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to fetch URL: {url}")
        return ""
    soup = BeautifulSoup(response.content, "html.parser")
    return soup.get_text(separator=" ", strip=True)

def main():
    df = pd.read_csv("jobs.csv")

    # Add new columns for extracted details
    df["salary"] = None
    df["experience"] = None
    df["technical_requirements"] = None
    df["education"] = None

    for idx, row in df.iterrows():
        job_url = row["job_detail_url"]
        print(f"Processing: {job_url}")
        job_text = fetch_job_description(job_url)
        if not job_text:
            continue
        details = extract_details_with_ollama(job_text)
        print(details)
        break
        df.at[idx, "salary"] = details.get("salary")
        df.at[idx, "experience"] = details.get("experience")
        df.at[idx, "technical_requirements"] = details.get("technical_requirements")
        df.at[idx, "education"] = details.get("education")

    df.to_csv("jobs_enriched.csv", index=False)
    print("Enriched CSV saved as jobs_enriched.csv")

if __name__ == "__main__":
    main()


Processing: https://www.linkedin.com/jobs/view/junior-data-scientist-at-louis-vuitton-4184288621?position=1&pageNum=0&refId=HgU79nN86L5X1M%2BurGd7EQ%3D%3D&trackingId=mw0LHARTIwNRAz%2F3Klz8VQ%3D%3D


In [ ]:
url = "https://www.linkedin.com/jobs/view/engineer-new-grad-at-nimblerx-4184351976?position=3&pageNum=0&refId=HgU79nN86L5X1M%2BurGd7EQ%3D%3D&trackingId=2jpxzuu7eV%2BjPWD9hS5MLg%3D%3D"
desc = fetch_job_description(url)

desc

'NimbleRx hiring Engineer (New Grad) in Redwood City, CA | LinkedIn LinkedIn and 3rd parties use essential and non-essential cookies to provide, secure, analyze and improve our Services, and to show you relevant ads (including professional and job ads ) on and off LinkedIn. Learn more in our Cookie Policy . Select Accept to consent or Reject to decline non-essential cookies for this use. You can update your choices at any time in your settings . Accept Reject Skip to main content LinkedIn Engineer (New Grad) in Milan Expand search This button displays the currently selected search type. When expanded it provides a list of search options that will switch the search inputs to match the current selection. Jobs People Learning Clear text Clear text Clear text Clear text Clear text Join now Sign in Engineer (New Grad) NimbleRx Redwood City, CA Apply Engineer (New Grad) NimbleRx Redwood City, CA 4 weeks ago Over 200 applicants See who NimbleRx has hired for this role Apply Save Report this j

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import re

def extract_details_with_ollama(job_text):
    url = "http://localhost:11434/api/generate"
    prompt = f"""
Extract the following details from the job description:
- Salary
- Years of experience required
- Technical requirements
- Education level requirements

Job description:
{job_text}

Output ONLY a JSON with keys: "salary", "experience", "technical_requirements", "education".

Example output:
{{
    "salary": "100000",
    "experience": "3 years",
    "technical_requirements": "Python, SQL",
    "education": "Bachelor's degree"
}}

"""
    payload = {"model": "llama3.2:1b", "prompt": prompt}

    response = requests.post(url, json=payload, stream=True)

    tokens = []
    for line in response.iter_lines():
        if line:
            try:
                token_data = json.loads(line.decode("utf-8"))
                token = token_data.get("response", "")
                tokens.append(token)
            except json.JSONDecodeError:
                # Skip improperly formatted lines
                continue

    final_answer = "".join(tokens)
    # Extract a valid JSON substring using regex
    match = re.search(r'\{.*\}', final_answer, re.DOTALL)
    if match:
        json_str = match.group(0)
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            print("Error: Extracted JSON substring is not valid JSON.")
            return {}
    else:
        print("Error: Could not extract valid JSON substring from LLM output.")
        return {}

def fetch_job_description(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to fetch URL: {url}")
        return ""
    soup = BeautifulSoup(response.content, "html.parser")
    return soup.get_text(separator=" ", strip=True)

def main():
    df = pd.read_csv("jobs.csv")

    # Add new columns for extracted details
    df["salary"] = None
    df["experience"] = None
    df["technical_requirements"] = None
    df["education"] = None

    for idx, row in df.iterrows():
        job_url = row["job_detail_url"]
        print(f"Processing: {job_url}")
        job_text = fetch_job_description(job_url)
        if not job_text:
            continue
        details = extract_details_with_ollama(job_text)
        print("==============")
        print("==============")
        print(details)
        print(details.get("salary"))
        print(details.get("experience"))
        print(details.get("technical_requirements"))
        print(details.get("education"))
        print("==============")
        print("==============")
        df.at[idx, "salary"] = details.get("salary")
        df.at[idx, "experience"] = details.get("experience")
        df.at[idx, "technical_requirements"] = details.get("technical_requirements")
        df.at[idx, "education"] = details.get("education")

    df.to_csv("jobs_enriched.csv", index=False)
    print("Enriched CSV saved as jobs_enriched.csv")

if __name__ == "__main__":
    main()


Processing: https://www.linkedin.com/jobs/view/junior-data-scientist-at-louis-vuitton-4184288621?position=1&pageNum=0&refId=HgU79nN86L5X1M%2BurGd7EQ%3D%3D&trackingId=mw0LHARTIwNRAz%2F3Klz8VQ%3D%3D
Processing: https://www.linkedin.com/jobs/view/junior-business-analyst-at-morgan-stanley-4182955414?position=2&pageNum=0&refId=HgU79nN86L5X1M%2BurGd7EQ%3D%3D&trackingId=grOGjHUdO%2BpamfGj0JbBug%3D%3D
Processing: https://www.linkedin.com/jobs/view/engineer-new-grad-at-nimblerx-4184351976?position=3&pageNum=0&refId=HgU79nN86L5X1M%2BurGd7EQ%3D%3D&trackingId=2jpxzuu7eV%2BjPWD9hS5MLg%3D%3D
Error: Extracted JSON substring is not valid JSON.
Processing: https://www.linkedin.com/jobs/view/master-data-coordinator-at-poppi-4183554421?position=4&pageNum=0&refId=HgU79nN86L5X1M%2BurGd7EQ%3D%3D&trackingId=SsLxu7Xciwdm5KnDBohCWQ%3D%3D
Processing: https://www.linkedin.com/jobs/view/senior-bi-analyst-at-safeway-4182900305?position=5&pageNum=0&refId=HgU79nN86L5X1M%2BurGd7EQ%3D%3D&trackingId=P0U4oIC030q8Y%2FAZH

KeyboardInterrupt: 